In [48]:
from astropy.io import ascii
from astropy.table import Table
import numpy as np

In [49]:
# Declerations
failed_cdfs_dir = '../ZFOURGE/data/cdfs_cigale_failed_ids' # List of failed galaxies by ZFOURGE ID, seperated by \n
cdfs_dir = '../ZFOURGE/data/cdfs/cdfs.v1.6.11.cat' # ZFOURGE CDFS field photometry data
cdfs_z_dir = '../ZFOURGE/data/cdfs/cdfs.v1.6.9.zout' # Output of Eazy, contains redshift data
filter_match = '../ZFOURGE/data/filter_match_table' # An ASCII table with the ZFOURGE filter name in 1st collumn, and filter transmission file name in 2nd col.
output_dir = '../ZFOURGE/data/working_sample' # Directory for output ascii table

output_failed_galaxies = True # True: will output all galaxies with ids found in list of failed ids.
only_zfourge_filters = False # True: will only use j1, j2, j3, hs, hl, and ks bands. False: uses most avalbile bands
add_random_successful_galaxy = False # True: will add a randomly selected successful galaxy to output
use_z_spec = True # True: will replace redshift in output with z_spec if value != -99.0
manual_galaxy_ids = [] # Optional list of manually selected galaxy ids to add to output. Expected to be non-failed galxies.

In [50]:
# Read list of failed ids, read cdfs photometry data, copy failed galaxies to new table.
failed_cdfs_table = Table.read(failed_cdfs_dir, format='ascii.commented_header')
failed_cdfs = np.array(failed_cdfs_table['__id'])
del(failed_cdfs_table)

cdfs_table = Table.read(cdfs_dir, format='ascii')
failed_cdfs_table = cdfs_table[:0].copy() # copies column headers of cdfs table, but no data (rows).

In [51]:
# Adds failed galxies when true
if output_failed_galaxies == True:
    for i in failed_cdfs:
        failed_cdfs_table.add_row(cdfs_table[i-1]) # row 0 of table is ID number 1.
        
# Adds manually selected galaxies when provided
for i in manual_galaxy_ids:
    failed_cdfs_table.add_row(cdfs_table[i-1])
    if i in failed_cdfs:
        print("Manually selected galaxy is in list of galaxies. This galaxy may appear twice in output.")

# Adds random galaxy not already included when true
if add_random_successful_galaxy == True:
    i = int(np.random.rand()*len(cdfs_table)) # i <- [0, 30911)
    while i+1 in failed_cdfs_table['id']: # +1 b/c row number (i) is 1 less than id number.
        i = int(np.random.rand()*len(cdfs_table))
    failed_cdfs_table.add_row(cdfs_table[i-1])

In [52]:
# Access and parse redshift data
cdfs_z_table = Table.read(cdfs_z_dir, format='ascii.commented_header')
failed_cdfs_z_table = cdfs_z_table[:0].copy()

for i in failed_cdfs_table['id']:
    failed_cdfs_z_table.add_row(cdfs_z_table[i-1])

In [53]:
# Cell defines output table and adds id and redshift columns
output = Table()
filter_table = Table.read(filter_match, format='ascii.commented_header')

output.add_columns([failed_cdfs_table['id'],failed_cdfs_z_table['z_peak']], names = ['id','redshift'])

In [54]:
# Cell adds FourStar filters only
if only_zfourge_filters == True:
    for i in range(14,26): # indexes of FourStar filters in filter_table
        output.add_column(failed_cdfs_table[filter_table['ZFOURGE'][i]], name = filter_table['CIGALE'][i])

In [55]:
# Cell adds (mostly) all filters
if only_zfourge_filters == False:
    for i in range(2, len(filter_table)): # goes through every filter in the match table
        output.add_column(failed_cdfs_table[filter_table['ZFOURGE'][i]], name = filter_table['CIGALE'][i])

In [56]:
# replaces photo z with z_spec if avalible
for i in range(len(output)):
    if cdfs_z_table['z_spec'][output['id'][i]-1] > -99.0:
        output['redshift'][i] = cdfs_z_table['z_spec'][output['id'][i]-1]

In [57]:
# Convert units of flux to mJy, and sets -99 to NaN
for i in output.columns[2:]:
    for j in range(len(i)):
        if output[i][j] == -99:
            output[i][j] = np.nan
    output[i] = output[i] * 0.3631 / 1000

In [58]:
output.write(output_dir, format='ascii', overwrite=True)

In [59]:
output

id,redshift,hst-ACS_update_sep07-wfc_f435w_t81,hst-ACS_update_sep07-wfc_f435w_t81_err,hst-ACS_update_sep07-wfc_f775w_t81,hst-ACS_update_sep07-wfc_f775w_t81_err,ESO-VIMOS-R,ESO-VIMOS-R_err,ESO-vimos_u,ESO-vimos_u_err,hst-ACS_update_sep07-wfc_f606w_t81,hst-ACS_update_sep07-wfc_f606w_t81_err,hst-ACS_update_sep07-wfc_f850lp_t81,hst-ACS_update_sep07-wfc_f850lp_t81_err,FOURSTAR-Hshort_cam_optics_sky,FOURSTAR-Hshort_cam_optics_sky_err,FOURSTAR-Hlong_cam_optics_sky,FOURSTAR-Hlong_cam_optics_sky_err,FOURSTAR-J1_cam_optics_sky,FOURSTAR-J1_cam_optics_sky_err,FOURSTAR-J2_cam_optics_sky,FOURSTAR-J2_cam_optics_sky_err,FOURSTAR-J3_cam_optics_sky,FOURSTAR-J3_cam_optics_sky_err,FOURSTAR-Ks_cam_optics_sky,FOURSTAR-Ks_cam_optics_sky_err,VLT-hawki_k_ETC,VLT-hawki_k_ETC_err,LCO-FourStar.NB118,LCO-FourStar.NB118_err,LCO-FourStar.NB209,LCO-FourStar.NB209_err,hst-wfc3-IR-f098m,hst-wfc3-IR-f098m_err,hst-wfc3-IR-f105w,hst-wfc3-IR-f105w_err,hst-wfc3-IR-f125w,hst-wfc3-IR-f125w_err,hst-wfc3-IR-f140w,hst-wfc3-IR-f140w_err,hst-wfc3-IR-f160w,hst-wfc3-IR-f160w_err,hst-wfc3-UVIS-f814w,hst-wfc3-UVIS-f814w_err,Subaru_MB-IA484,Subaru_MB-IA484_err,Subaru_MB-IA527,Subaru_MB-IA527_err,Subaru_MB-IA574,Subaru_MB-IA574_err,Subaru_MB-IA598,Subaru_MB-IA598_err,Subaru_MB-IA624,Subaru_MB-IA624_err,Subaru_MB-IA651,Subaru_MB-IA651_err,Subaru_MB-IA679,Subaru_MB-IA679_err,Subaru_MB-IA738,Subaru_MB-IA738_err,Subaru_MB-IA768,Subaru_MB-IA768_err,Subaru_MB-IA797,Subaru_MB-IA797_err,Subaru_MB-IA856,Subaru_MB-IA856_err,ESO-WFI-V89_843,ESO-WFI-V89_843_err,ESO-WFI-Rc162_844,ESO-WFI-Rc162_844_err,ESO-wfi_BB_U38_ESO841,ESO-wfi_BB_U38_ESO841_err,WIRCam-cfh8302_Ks,WIRCam-cfh8302_Ks_err,IRAC-irac_tr1_2004-08-09,IRAC-irac_tr1_2004-08-09_err,IRAC-irac_tr2_2004-08-09,IRAC-irac_tr2_2004-08-09_err,IRAC-irac_tr3_2004-08-09,IRAC-irac_tr3_2004-08-09_err,IRAC-irac_tr4_2004-08-09,IRAC-irac_tr4_2004-08-09_err
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
4,3.812,nan,nan,nan,nan,nan,nan,0.20598889835832,3.6513618114176e-05,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.188547510042,0.00011483216704373999,2.0150260512484,0.00012076342413446,4.515529101419999,0.000303144270789,1.9468520912885,0.00012008641848378998,1.8604006275613,0.00013434078685066,1.5748762969694998,0.00010567575477490999,1.2391130847359997,0.00013275072177024,1.8686636496,0.00014475805126992,1.5227650934457,0.00036752823463277994,1.3649675696995,0.0004154719554375,1.6025305688461,0.0005602652117214999,0.7691163441572999,0.00010924282702580999,0.8242095670687999,0.00010133491878416,0.22796917403294994,0.00038045637625555,0.4662318492692,0.0007159231349494001,nan,nan,nan,nan,nan,nan,nan,nan
64,7.3259,nan,nan,nan,nan,nan,nan,0.0009225410237400001,7.349137057528e-06,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.013918155202931999,2.2918953247255995e-05,0.021285899131148,2.4060144776005996e-05,0.025891025307119998,6.0446180726399995e-05,0.03824966720102,2.3917134155541e-05,0.03705378155617,2.6803153879186e-05,0.036601200935049996,2.1089051012841e-05,0.04773158064639999,2.64873519896e-05,0.02624015380008,2.8879668524784e-05,0.025429559941265997,7.328896618095e-05,0.030202411890819996,8.288614561239999e-05,0.025218382335628994,0.00011166690448442998,0.001586118631